In [ ]:
from pathlib import Path

# Create dir structure
base_img_dir = Path('mnist_images')

for sub in ['test', 'training']:
    (base_img_dir / sub).mkdir(parents=True, exist_ok=True)

weeks_dir = base_img_dir / 'weeks'
for i in range(1, 8):
    (weeks_dir / f'week {i}').mkdir(parents=True, exist_ok=True)

In [18]:
import os
import tensorflow as tf
import numpy as np
from pathlib import Path
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten
from sklearn.model_selection import train_test_split
from PIL import Image

# Get the week directory from the environment variable
week = os.getenv('WEEK', 'week 1')  # Default to 'week 1'
week_dir = Path(f'mnist_images/weeks/{week}')
train_dir = Path('mnist_images/training')
test_dir = Path('mnist_images/test')
print(f"Using week directory: {week_dir}")

train_dir.mkdir(parents=True, exist_ok=True)
test_dir.mkdir(parents=True, exist_ok=True)

image_paths = sorted(list(week_dir.glob('*.png')))
images = []
labels = []

for img_path in image_paths:
    img = Image.open(img_path).convert('L')  # Ensure grayscale
    img_array = np.array(img) / 255.0        # Normalize
    images.append(img_array)
    # Parse label from filename, for example '00001_label_7.png'
    label = int(img_path.stem.split('_')[-1])
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

x_train, x_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=17, stratify=labels
)

for idx, (img, label) in enumerate(zip(x_train, y_train)):
    img_uint8 = (img * 255).astype(np.uint8)  # Convert back to uint8 for saving
    img_path = train_dir / f'{idx:05d}_label_{label}.png'
    Image.fromarray(img_uint8).save(img_path)

for idx, (img, label) in enumerate(zip(x_test, y_test)):
    img_uint8 = (img * 255).astype(np.uint8)
    img_path = test_dir / f'{idx:05d}_label_{label}.png'
    Image.fromarray(img_uint8).save(img_path)

print(f"x_train shape: {x_train.shape}, x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

model = Sequential([
    Input(shape=(28, 28)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=2)

# Save the model
model.save('mnist_model.h5')

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Accuracy: {test_acc * 100:.2f}%")

Using week directory: mnist_images/weeks/week 2
x_train shape: (20520, 28, 28), x_test shape: (5131, 28, 28)
y_train shape: (20520,), y_test shape: (5131,)
Epoch 1/5
642/642 - 11s - 17ms/step - accuracy: 0.8744 - loss: 0.4451
Epoch 2/5
642/642 - 10s - 15ms/step - accuracy: 0.9137 - loss: 0.3094
Epoch 3/5
642/642 - 9s - 14ms/step - accuracy: 0.9203 - loss: 0.2935
Epoch 4/5
642/642 - 7s - 12ms/step - accuracy: 0.9204 - loss: 0.2866
Epoch 5/5
642/642 - 6s - 9ms/step - accuracy: 0.9223 - loss: 0.2788


161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9073 - loss: 0.3494
Accuracy: 90.90%
